In [30]:
%matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 22: Monkey Map ---</h2><p>The monkeys take you on a surprisingly easy trail through the jungle. They're even going in roughly the right direction according to your handheld device's Grove Positioning System.</p>
<p>As you walk, the monkeys explain that the grove is protected by a <em>force field</em>. To pass through the force field, you have to enter a password; doing so involves tracing a specific <em>path</em> on a strangely-shaped board.</p>
<p>At least, you're pretty sure that's what you have to do; the elephants aren't exactly fluent in monkey.</p>
<p>The monkeys give you notes that they took when they last saw the password entered (your puzzle input).</p>
<p>For example:</p>
<pre><code>        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
</code></pre>

<p>The first half of the monkeys' notes is a <em>map of the board</em>. It is comprised of a set of <em>open tiles</em> (on which you can move, drawn <code>.</code>) and <em>solid walls</em> (tiles which you cannot enter, drawn <code>#</code>).</p>
<p>The second half is a description of <em>the path you must follow</em>. It consists of alternating numbers and letters:</p>
<ul>
<li>A <em>number</em> indicates the <em>number of tiles to move</em> in the direction you are facing. If you run into a wall, you stop moving forward and continue with the next instruction.</li>
<li>A <em>letter</em> indicates whether to turn 90 degrees <em>clockwise</em> (<code>R</code>) or <em><span title="Or &quot;anticlockwise&quot;, if you're anti-counterclockwise.">counterclockwise</span></em> (<code>L</code>). Turning happens in-place; it does not change your current tile.</li>
</ul>
<p>So, a path like <code>10R5</code> means "go forward 10 tiles, then turn clockwise 90 degrees, then go forward 5 tiles".</p>
<p>You begin the path in the leftmost open tile of the top row of tiles. Initially, you are facing <em>to the right</em> (from the perspective of how the map is drawn).</p>
<p>If a movement instruction would take you off of the map, you <em>wrap around</em> to the other side of the board. In other words, if your next tile is off of the board, you should instead look in the direction opposite of your current facing as far as you can until you find the opposite edge of the board, then reappear there.</p>
<p>For example, if you are at <code>A</code> and facing to the right, the tile in front of you is marked <code>B</code>; if you are at <code>C</code> and facing down, the tile in front of you is marked <code>D</code>:</p>
<pre><code>        ...#
        .#..
        #...
        ....
...#.<em>D</em>.....#
........#...
<em>B</em>.#....#...<em>A</em>
.....<em>C</em>....#.
        ...#....
        .....#..
        .#......
        ......#.
</code></pre>
<p>It is possible for the next tile (after wrapping around) to be a <em>wall</em>; this still counts as there being a wall in front of you, and so movement stops before you actually wrap to the other side of the board.</p>
<p>By drawing the <em>last facing you had</em> with an arrow on each tile you visit, the full path taken by the above example looks like this:</p>
<pre><code>        &gt;&gt;v#    
        .#v.    
        #.v.    
        ..v.    
...#...v..v#    
&gt;&gt;&gt;v...<em>&gt;</em>#.&gt;&gt;    
..#v...#....    
...&gt;&gt;&gt;&gt;v..#.    
        ...#....
        .....#..
        .#......
        ......#.
</code></pre>
<p>To finish providing the password to this strange input device, you need to determine numbers for your final <em>row</em>, <em>column</em>, and <em>facing</em> as your final position appears from the perspective of the original map. Rows start from <code>1</code> at the top and count downward; columns start from <code>1</code> at the left and count rightward. (In the above example, row 1, column 1 refers to the empty space with no tile on it in the top-left corner.) Facing is <code>0</code> for right (<code>&gt;</code>), <code>1</code> for down (<code>v</code>), <code>2</code> for left (<code>&lt;</code>), and <code>3</code> for up (<code>^</code>). The <em>final password</em> is the sum of 1000 times the row, 4 times the column, and the facing.</p>
<p>In the above example, the final row is <code>6</code>, the final column is <code>8</code>, and the final facing is <code>0</code>. So, the final password is 1000 * 6 + 4 * 8 + 0: <code><em>6032</em></code>.</p>
<p>Follow the path given in the monkeys' notes. <em>What is the final password?</em></p>
</article>


In [ ]:
from copy import deepcopy


s = """
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
"""

tests = [
    {
        "name": "Example 1",
        "s": s,
        "expected": 6032,
    },
]


class MonkeyMap:
    def __init__(self, s: str) -> None:
        map, key = re.split(r"\n\n", s)

        self.monkey_map, self.rows, self.cols = self._create_map(map)

        self.path = [
            int(ch) if ch.isdecimal() else ch
            for ch in re.findall(rf"([RL]|[0-9]+)", key.strip())
        ]

    def walk_path(self, debug=False) -> tuple[int, int, int]:
        r = min(m for m, _ in self.cols.values())
        c = self.rows[r][0]
        f = 0

        if debug:
            direction = lambda f: [">", "v", "<", "^"][f]
            vivisted = {(r, c): direction(f)}

        while self.monkey_map[(r, c)] != ".":
            if c > self.rows[r][1]:
                r += 1
                c = self.rows[r][0]
            else:
                c += 1

        for v in self.path:
            match v:
                case "R":
                    f = (f + 1) % 4
                case "L":
                    f = (f - 1) % 4
                case int():
                    if f == 0:  # >
                        d = 1
                        while d <= v:
                            if (r, c + 1) in self.monkey_map:
                                if self.monkey_map[(r, c + 1)] == "#":
                                    break
                                c += 1
                            else:
                                if self.monkey_map[(r, self.rows[r][0])] == "#":
                                    break
                                c = self.rows[r][0]
                            d += 1

                            if debug:
                                vivisted[(r, c)] = direction(f)

                    elif f == 1:  # v
                        d = 1
                        while d <= v:
                            if (r + 1, c) in self.monkey_map:
                                if self.monkey_map[(r + 1, c)] == "#":
                                    break
                                r += 1
                            else:
                                if self.monkey_map[(self.cols[c][0], c)] == "#":
                                    break
                                r = self.cols[c][0]
                            d += 1

                            if debug:
                                vivisted[(r, c)] = direction(f)

                    elif f == 2:  # <
                        d = 1
                        while d <= v:
                            if (r, c - 1) in self.monkey_map:
                                if self.monkey_map[(r, c - 1)] == "#":
                                    break
                                c -= 1
                            else:
                                if self.monkey_map[(r, self.rows[r][1])] == "#":
                                    break
                                c = self.rows[r][1]
                            d += 1

                            if debug:
                                vivisted[(r, c)] = direction(f)

                    else:  # ^
                        d = 1
                        while d <= v:
                            if (r - 1, c) in self.monkey_map:
                                if self.monkey_map[(r - 1, c)] == "#":
                                    break
                                r -= 1
                            else:
                                if self.monkey_map[(self.cols[c][1], c)] == "#":
                                    break
                                r = self.cols[c][1]
                            d += 1

                            if debug:
                                vivisted[(r, c)] = direction(f)
                    if debug:
                        print(self.str(vivisted))

        return 1_000 * (r + 1) + 4 * (c + 1) + f

    @classmethod
    def _create_map(
        cls, map: str
    ) -> tuple[dict[tuple[int, int], str], dict[int][list[int]]]:
        map = [list(l) for l in map.splitlines() if l]
        monkey_map = {}
        rows = defaultdict(lambda: [inf, -inf])
        cols = defaultdict(lambda: [inf, -inf])

        for r, row in enumerate(map):
            for c, col in enumerate(row):
                if col != " ":
                    monkey_map[(r, c)] = col
                    rows[r][0] = min(rows[r][0], c)
                    rows[r][1] = max(rows[r][1], c)
                    cols[c][0] = min(cols[c][0], r)
                    cols[c][1] = max(cols[c][1], r)

        return monkey_map, rows, cols

    def str(self, vistited=None) -> str:
        mm = deepcopy(self.monkey_map)
        for k, v in vistited.items():
            mm[k] = v

        rows = len(self.rows)
        cols = max(m[1] for m in self.rows.values()) + 1
        mp = [[mm.get((r, c), " ") for c in range(cols)] for r in range(rows)]
        return f'{"\n".join("".join(c for c in r) for r in mp)}\n\n'


@test(tests=tests)
def test_part_I(s: str) -> int:
    return MonkeyMap(s).walk_path()


Test Example 1 passed, for test_part_I.
Success


In [32]:
with open("../input/day22.txt") as f:
    puzzle = f.read()

print(f"Part I: {MonkeyMap(puzzle).walk_path():,}")

Part I: 186,128


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>186128</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you reach the force field, you think you hear some Elves in the distance. Perhaps they've already arrived?</p>
<p>You approach the strange <em>input device</em>, but it isn't quite what the monkeys drew in their notes. Instead, you are met with a large <em>cube</em>; each of its six faces is a square of 50x50 tiles.</p>
<p>To be fair, the monkeys' map <em>does</em> have six 50x50 regions on it. If you were to <em>carefully fold the map</em>, you should be able to shape it into a cube!</p>
<p>In the example above, the six (smaller, 4x4) faces of the cube are:</p>
<pre><code>        1111
        1111
        1111
        1111
222233334444
222233334444
222233334444
222233334444
        55556666
        55556666
        55556666
        55556666
</code></pre>
<p>You still start in the same position and with the same facing as before, but the <em>wrapping</em> rules are different. Now, if you would walk off the board, you instead <em>proceed around the cube</em>. From the perspective of the map, this can look a little strange. In the above example, if you are at A and move to the right, you would arrive at B facing down; if you are at C and move down, you would arrive at D facing up:</p>
<pre><code>        ...#
        .#..
        #...
        ....
...#.......#
........#..<em>A</em>
..#....#....
.<em>D</em>........#.
        ...#..<em>B</em>.
        .....#..
        .#......
        ..<em>C</em>...#.
</code></pre>
<p>Walls still block your path, even if they are on a different face of the cube. If you are at E facing up, your movement is blocked by the wall marked by the arrow:</p>
<pre><code>        ...#
        .#..
     <em>--&gt;#</em>...
        ....
...#..<em>E</em>....#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.
</code></pre>
<p>Using the same method of drawing the <em>last facing you had</em> with an arrow on each tile you visit, the full path taken by the above example now looks like this:</p>
<pre><code>        &gt;&gt;v#    
        .#v.    
        #.v.    
        ..v.    
...#..<em>^</em>...v#    
.&gt;&gt;&gt;&gt;&gt;^.#.&gt;&gt;    
.^#....#....    
.^........#.    
        ...#..v.
        .....#v.
        .#v&lt;&lt;&lt;&lt;.
        ..v...#.
</code></pre>
<p>The final password is still calculated from your final position and facing from the perspective of the map. In this example, the final row is <code>5</code>, the final column is <code>7</code>, and the final facing is <code>3</code>, so the final password is 1000 * 5 + 4 * 7 + 3 = <code><em>5031</em></code>.</p>
<p>Fold the map into a cube, <em>then</em> follow the path given in the monkeys' notes. <em>What is the final password?</em></p>
</article>


In [33]:
# print(f"PartI II: {MonkeysII(puzzle).symbolic('root')}")

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>3769668716709</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
